In [1]:
import json
import os


In [2]:
# Reading in results
### bbox format: (x1, y1, x2, y2)

with open(os.path.join("./results", "data_dict.json"), "r") as f:
    res_obj = json.load(f)

In [3]:
# Reading in ground truth
### bbox format: (x1, y1, w, h)

dir_name = "far_shah_b1-b3_EOI_split"

with open(os.path.join("./", dir_name, "test/result.json"), "r") as f:
    coco_gt_obj = json.load(f)

In [4]:
res_obj[0]

{'file_name': '/home/dtron2_user/ls_dtron2_full/model/far_shah_b1-b3_EOI_split/test/images/ZF4ZoJhxQjlNSu7V-serenohotels_ss.png',
 'height': 1080,
 'width': 1920,
 'annotations': [{'iscrowd': 0,
   'bbox': [1095.8163, 970.0453, 1252.8141, 1011.1241],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [1095.2361, 858.6618, 1253.1652, 901.2455],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [861.5819, 2.8884, 1059.9102, 68.0269],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [1095.2179, 912.0303, 1252.0514, 956.3156],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [42.4671, 908.2764, 107.5965, 975.386],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [1212.4924, 1049.0056, 1295.7235, 1079.5737],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [1822.4149, 15.0026, 1911.9615, 40.7503],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [1467.7878

In [5]:
coco_gt_obj["annotations"][0]

{'id': 30144,
 'image_id': 2114,
 'category_id': 1,
 'segmentation': [],
 'bbox': [1872.5691699604743,
  4.268774703557313,
  41.26482213438749,
  36.996047430830046],
 'ignore': 0,
 'iscrowd': 0,
 'area': 1526.635316908565}

In [6]:
# Group By Image

imgs_gt_pred_list = {}

for img in coco_gt_obj["images"]:
    fn1 = img["file_name"][2:]
    img_id = img["id"]
    
    gt_list = []

    for ann1 in coco_gt_obj["annotations"]:
        if img_id == ann1["image_id"]:
            x1 = ann1["bbox"][0]
            y1 = ann1["bbox"][1]
            w = ann1["bbox"][2]
            h = ann1["bbox"][3]
            gt_list.append([x1, y1, x1 + w, y1 + h])
    
    pred_list = []
    
    for res in res_obj:
        fn2 = os.path.basename(res["file_name"])
        if fn1 == fn2:
            for ann2 in res["annotations"]:
                pred_list.append(ann2["bbox"])
    
    imgs_gt_pred_list[fn1] = [gt_list, pred_list]
            



In [7]:
keys = list(imgs_gt_pred_list.keys())


In [8]:
def calculate_iou(boxA, boxB):
    
    # Determine the coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    # Compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)
    
    # Compute the area of both the prediction and true bounding boxes
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    
    # Compute the area of union
    unionArea = boxAArea + boxBArea - interArea
    
    # Compute the Intersection over Union by dividing the intersection area by the union area
    iou = interArea / unionArea
    
    return iou

In [35]:
thold = .95

In [36]:
iou_scores = []
for key in keys:
    gts = imgs_gt_pred_list[key][0]
    preds = imgs_gt_pred_list[key][1]
    per_img_scores = []
    for i, pred in enumerate(preds): 
        for j, gt in enumerate(gts):
            iou = calculate_iou(pred, gt)
            if iou >= thold:
                per_img_scores.append(iou)
                break
        if iou < thold: 
            per_img_scores.append(iou)
    iou_scores.append([len(gts), len(preds), per_img_scores])

In [37]:
conf_mat = []
for img in iou_scores:
    tp = len([x for x in img[2] if x > thold])
    num_gt = img[0]
    num_pred = img[1]
    fp = num_pred - tp
    fn = num_gt - tp
    if fn < 0:
        fn = 0
    conf_mat.append([tp, fp, fn])
    

In [38]:
iou_scores_flat = []
for img in iou_scores:
    for item in img[-1]:
        iou_scores_flat.append(item)
corr = len([x for x in iou_scores_flat if x > thold])
accuracy = corr/len(iou_scores_flat)
accuracy

0.003979131665045539

In [39]:
tp_tot = 0
fp_tot = 0
fn_tot = 0

for img in conf_mat:
    tp_tot = tp_tot + img[0]
    fp_tot = fp_tot + img[1]
    tn_tot = 0
    fn_tot = fn_tot + img[2]
    

In [40]:
precision = tp_tot/(tp_tot + fp_tot)
recall = tp_tot/(tp_tot + fn_tot)
f1_score = 2 * (precision * recall) / (precision + recall)
print(precision)
print(recall)
print(f1_score)

0.003979131665045539
0.006669630947087594
0.004984492689410722
